In [1]:
import pandas as pd
nfl_players = pd.read_csv('all_predictions.csv')

In [2]:
nfl_players.head()

,Name,high,low,point,FantPos,prediction
0,A.J. Derby,36.0,0.0,6.0,TE,66.902552
1,A.J. Green,160.7,12.7,96.0,WR,106.126280
2,A.J. McCarron,301.4,0.0,24.0,0,81.901397
3,Aaron Jones,168.3,10.8,84.0,RB,139.796658
4,Aaron Rodgers,354.1,222.9,279.0,QB,191.763935


In [3]:
import numpy as np
nfl_players['mean_prediction'] = nfl_players.apply(lambda x: np.mean(x[['low', 'point', 'prediction']]), axis=1)

In [4]:
nfl_players.sort_values('mean_prediction', ascending=False).head()

,Name,high,low,point,FantPos,prediction,mean_prediction
522,Russell Wilson,337.9,188.9,264.5,QB,279.261277,244.220426
86,Cam Newton,337.9,188.9,269.0,QB,272.902270,243.600757
4,Aaron Rodgers,354.1,222.9,279.0,QB,191.763935,231.221312
211,Drew Brees,354.1,222.9,269.0,QB,201.652090,231.184030
577,Tom Brady,337.9,162.8,263.0,QB,227.616667,217.805556


In [ ]:
import tqdm 
class DraftState:
    def __init__(self, rosters, turns, freeagents, playerjm=None):
        self.rosters = rosters
        self.turns = turns
        self.freeagents = freeagents
        self.playerJustMoved = playerjm
        
class NflPlayer:
    def __init__(self, name, position, points, high, low):
        self.name = name
        self.position = position
        self.points = points
        self.high = high
        self.low = low
        
    def __repr__(self):
        return "|".join([self.name, self.position, str(self.points)])

    
import numpy as np

def GetResult(self, playerjm):
    """ Get the game result from the viewpoint of playerjm.
    """
    if playerjm is None: return 0
    
    pos_wgts = {
        ("QB"): [.5],
        ("WR"): [.5, .5, .5, .5],
        ("RB"): [.5, .5, .5, .5],
        ("TE"): [.5, .5],
        ("RB", "WR", "TE"): [.5, .5, .5]
    }

    result = 0
    # map the drafted players to the weights
    for p in self.rosters[playerjm]:
        max_wgt, _, max_pos, old_wgts = max(
            ((wgts[0], -len(lineup_pos), lineup_pos, wgts) for lineup_pos, wgts in pos_wgts.items()
                if p.position in lineup_pos),
            default=(0, 0, (), []))
        if max_wgt > 0:
            
            random_points = np.random.randint(p.low, p.high+1, 1)[0]
            result += max_wgt * random_points
            old_wgts.pop(0)
            if not old_wgts:
                pos_wgts.pop(max_pos)
                
    # map the remaining weights to the top three free agents
    for pos, wgts in pos_wgts.items():
        result += np.mean([np.random.randint(p.low, p.high+1, 1)[0] for p in self.freeagents if p.position in pos][:5]) 
    return result
        
DraftState.GetResult = GetResult


def GetMoves(self):
    """ Get all possible moves from this state.
    """
    pos_max = {"QB": 1, "WR": 6, "RB": 6, "TE": 2}

    if len(self.turns) == 0: return []

    roster_positions = np.array([p.position for p in self.rosters[self.turns[0]]], dtype=str)
    moves = [pos for pos, max_ in pos_max.items() if np.sum(roster_positions == pos) < max_]
    return moves

DraftState.GetMoves = GetMoves


def DoMove(self, move):
    """ Update a state by carrying out the given move.
        Must update playerJustMoved.
    """
    player = next(p for p in self.freeagents if p.position == move)
    self.freeagents.remove(player)
    rosterId = self.turns.pop(0)
    self.rosters[rosterId].append(player)
    self.playerJustMoved = rosterId
    
DraftState.DoMove = DoMove


def Clone(self):
    """ Create a deep clone of this game state.
    """
    rosters = list(map(lambda r: r[:], self.rosters))
    st = DraftState(rosters, self.turns[:], self.freeagents[:],
            self.playerJustMoved)
    return st

DraftState.Clone = Clone

In [ ]:

# This is a very simple implementation of the UCT Monte Carlo Tree Search algorithm in Python 2.7.
# The function UCT(rootstate, itermax, verbose = False) is towards the bottom of the code.
# It aims to have the clearest and simplest possible code, and for the sake of clarity, the code
# is orders of magnitude less efficient than it could be made, particularly by using a 
# state.GetRandomMove() or state.DoRandomRollout() function.
# 
# Written by Peter Cowling, Ed Powley, Daniel Whitehouse (University of York, UK) September 2012.
# 
# Licence is granted to freely use and distribute for any sensible/legal purpose so long as this comment
# remains in any distributed code.
# 
# For more information about Monte Carlo Tree Search check out our web site at www.mcts.ai

from math import *
import random

class Node:
    """ A node in the game tree. Note wins is always from the viewpoint of playerJustMoved.
        Crashes if state not specified.
    """
    def __init__(self, move = None, parent = None, state = None):
        self.move = move # the move that got us to this node - "None" for the root node
        self.parentNode = parent # "None" for the root node
        self.childNodes = []
        self.wins = 0
        self.visits = 0
        self.untriedMoves = state.GetMoves() # future child nodes
        self.playerJustMoved = state.playerJustMoved # the only part of the state that the Node needs later
        
    def UCTSelectChild(self):
        """ Use the UCB1 formula to select a child node. Often a constant UCTK is applied so we have
            lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits to vary the amount of
            exploration versus exploitation.
        """
        UCTK = 200
        s = sorted(self.childNodes, key = lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits))[-1]
        return s
    
    def AddChild(self, m, s):
        """ Remove m from untriedMoves and add a new child node for this move.
            Return the added child node
        """
        n = Node(move = m, parent = self, state = s)
        self.untriedMoves.remove(m)
        self.childNodes.append(n)
        return n
    
    def Update(self, result):
        """ Update this node - one additional visit and result additional wins. result must be from the viewpoint of playerJustmoved.
        """
        self.visits += 1
        self.wins += result


def UCT(rootstate, itermax, verbose = False):
    """ Conduct a UCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
    """

    rootnode = Node(state = rootstate)

    for i in tqdm.tqdm(range(itermax)):
        node = rootnode
        state = rootstate.Clone()

        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.UCTSelectChild()
            state.DoMove(node.move)

        # Expand
        if node.untriedMoves != []: # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves) 
            state.DoMove(m)
            node = node.AddChild(m,state) # add child and descend tree

        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function
        while state.GetMoves() != []: # while state is non-terminal
            state.DoMove(random.choice(state.GetMoves()))

        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            node.Update(state.GetResult(node.playerJustMoved)) # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode

    return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move # return the move that was most visited

In [ ]:
nfl_players = nfl_players[['Name','FantPos', 'mean_prediction', 'high', 'low']]


In [ ]:
nfl_players = nfl_players[nfl_players.mean_prediction > 75].reset_index(drop=True)

In [ ]:
nfl_players = nfl_players.sort_values('mean_prediction', ascending=False)

In [ ]:
nfl_players.head()

,Name,FantPos,mean_prediction,high,low
303,Russell Wilson,QB,244.220426,337.9,188.9
44,Cam Newton,QB,243.600757,337.9,188.9
1,Aaron Rodgers,QB,231.221312,354.1,222.9
118,Drew Brees,QB,231.184030,354.1,222.9
336,Tom Brady,QB,217.805556,337.9,162.8


In [ ]:
freeagents = [NflPlayer(*p) for p in nfl_players.itertuples(index=False, name=None)]

In [ ]:
import pandas as pd

num_competitors = 12
rosters = [[] for _ in range(num_competitors)] # empty rosters to start with

num_rounds = 15
turns = []
# generate turns by snake order
for i in range(num_rounds):
    turns += reversed(range(num_competitors)) if i % 2 else range(num_competitors)
    
state = DraftState(rosters, turns, freeagents)
iterations = 180000
while state.GetMoves() != []:
    move = UCT(state, iterations)
    state.DoMove(move)

 48%|████▊     | 85710/180000 [37:37<41:55, 37.49it/s]

In [ ]:
pd.DataFrame({"Team " + str(i + 1): r for i, r in enumerate(state.rosters)})

In [ ]:
for i in np.arange(0, len(state.rosters)):
    one_team = pd.Series(state.rosters[i])
    t = one_team.astype(str).str.split('|', expand=True)
    print({i + 1 : np.sum(t[2].astype(float))})